# BIFROST Bragg peak monitor

In [ ]:
from ess.bifrost.single_crystal import BifrostBraggPeakMonitorWorkflow
from ess.spectroscopy.types import *
from ess.bifrost.types import McStasDetectorData
from ess.bifrost.data import simulated_elastic_incoherent_with_phonon, tof_lookup_table_simulation
from ess.reduce.nexus.types import NeXusDetectorName
from ess.reduce.time_of_flight import DetectorLtotal
import scippnexus as snx
from ess.bifrost.single_crystal.types import *

In [ ]:
workflow = BifrostBraggPeakMonitorWorkflow()
workflow[Filename[SampleRun]] = simulated_elastic_incoherent_with_phonon()
workflow[TimeOfFlightLookupTableFilename] = tof_lookup_table_simulation()

workflow[QBinsParallel] = sc.linspace('Q_parallel', -3, 3, 200, unit='1/Å')
workflow[QBinsPerpendicular] = sc.linspace('Q_perpendicular', -3, 3, 200, unit='1/Å')

In [ ]:
# Load an inelastic detector as we don't have an actual bragg peak monitor in the file.
workflow[NeXusDetectorName] = "309_channel_9_5_triplet"


def assemble_detector_data_flatten(
        detector: CalibratedBeamline[RunType],
        data: NeXusData[snx.NXdetector, RunType],
) -> McStasDetectorData[RunType]:
    from ess.reduce.nexus.workflow import assemble_detector_data as assemble
    base = assemble(detector, data)

    # The monitor only has one pixel, so combine all pixels from the selected bank:
    monitor = base.bins.concat()
    monitor.coords['detector_number'] = sc.index(0)
    # Halfway between the bank and the sample:
    monitor.coords['position'] = base.coords['position'].mean() / 2
    return McStasDetectorData[RunType](monitor)


def detector_ltotal_from_straight_line_approximation(
        detector_beamline: DetectorData[RunType],
) -> DetectorLtotal[RunType]:
    from ess.reduce.time_of_flight import eto_to_tof
    return eto_to_tof.detector_ltotal_from_straight_line_approximation(
        detector_beamline  # type: ignore[arg-type]
    )


workflow.insert(assemble_detector_data_flatten)
workflow.insert(detector_ltotal_from_straight_line_approximation)

In [ ]:
workflow.visualize(QMap[SampleRun], graph_attr={"rankdir": "LR"})

In [ ]:
da = workflow.compute(QMap[SampleRun])

In [ ]:
da

In [ ]:
da.hist().plot(norm='log')

In [ ]:
da.bins.coords['Q'].min()

In [ ]:
da.bins.coords['Q'].max()

In [ ]:
da.bins.concat().bin(Q=sc.linspace('Q', 2.6, 2.7, 100, unit='1/Å')).bins.concat().hist(a3=180).plot()